In [ ]:
import pandas as pd
import ast
import re

In [ ]:
# Read the chordonomicon dataset
df = pd.read_csv('chordonomicon.csv')

# Read the mapping CSV file
chord_relations = pd.read_csv('chords_mapping.csv')
# Create a dictionary with keys the "chords" and values the "degrees"
chord_degrees = dict(zip(chord_relations['Chords'], chord_relations['Degrees']))
for key, value in chord_degrees.items():
    chord_degrees[key] = ast.literal_eval(value)
chord_notes = dict(zip(chord_relations['Chords'], chord_relations['Notes']))
for key, value in chord_notes.items():
    chord_notes[key] = ast.literal_eval(value)


In [ ]:
# remove inversions
df['chords'] = df['chords'].apply(lambda s: re.sub(r"/[^/]*$", "", s))


In [ ]:
# map the chords to notes and binary degree table
for progression in df['chords']:
    for chord in progression.split():
            if ">" not in chord:
                notes = chord_notes[chord]
                degrees = chord_degrees[chord]

In [ ]:
# pitch shifting

# Define musical elements
degrees_sharp = ['C', 'Cs', 'D', 'Ds', 'E', 'F', 'Fs', 'G', 'Gs', 'A', 'As', 'B']
degrees_flat = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
notes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
accs = ['b', 's']
all_notes_list = [note + acc for note in notes for acc in accs]

def convert_chord(chord, shift):
    """
    Convert chords in a progression based on the specified shift.

    Args:
        chord (str): Chord progression as a string.
        shift (int): The number of semitones to shift.

    Returns:
        tuple: Two lists of converted chord progressions.
    """
    converted_chord = []
    converted_chord_base = []
    chord_progression = chord
    for cchord in chord_progression:
        if ('>' in cchord):
            converted_chord.append(cchord)
            converted_chord_base.append(cchord)
        elif cchord == '':
            pass
        else:
            degree = 'next'
            rest = 'next'
            for note in all_notes_list:
                if cchord.startswith(note):
                    if 'sus' in cchord and 'ssus' not in cchord and 'bsus' not in cchord and '7sus' not in cchord:
                        break
                    else:
                        degree = note
                        degree = degree.replace("Bs",'C').replace('Cb',"B").replace('Es','F').replace('Fb','E')
                        rest = cchord.replace(note, '', 1)
                        break
                else:
                    for note in notes:
                        if cchord.startswith(note):
                            degree = note
                            degree = degree.replace("Bs",'C').replace('Cb',"B").replace('Es','F').replace('Fb','E')
                            rest = cchord.replace(note, '', 1)
                            break

            if degree in degrees_sharp:
                index = degrees_sharp.index(degree)
                next_index = (index + shift) % len(degrees_sharp)
                next_degree = degrees_sharp[next_index]
            elif degree in degrees_flat:
                index = degrees_flat.index(degree)
                next_index = (index + shift) % len(degrees_flat)
                next_degree = degrees_flat[next_index]
            else:
                # debug check
                next_degree = 'next'
                print(cchord)

            next_base = ''  # Initialize next_base here

            if '/' in cchord:
                base = cchord.split('/')[1]
                extension = rest.replace(base, '').replace('/', '')
                if base in degrees_sharp:
                    index = degrees_sharp.index(base)
                    next_index = (index + shift) % len(degrees_sharp)
                    next_base = degrees_sharp[next_index]
                elif base in degrees_flat:
                    index = degrees_flat.index(base)
                    next_index = (index + shift) % len(degrees_flat)
                    next_base = degrees_flat[next_index]

                next_base = next_base.replace("Bs",'C').replace('Cb',"B").replace('Es','F').replace('Fb','E')
                converted_chord_base.append(next_degree + extension + '/' + next_base)
                converted_chord.append(next_degree + extension)
                
            else:
                converted_chord.append(next_degree + rest)
                converted_chord_base.append(next_degree + rest)
    return converted_chord, converted_chord_base

# Create dictionaries to store shifted chord progressions
shift_lists = {}
shift_lists_base = {}

# Iterate over the specified range of shifts (1 to 12)
for shift in range(1, 12):  # Adjust the range as needed (e.g., 1 to 12 for multiple shifts)
    list_name = f"shift_{shift}"
    shift_lists[list_name] = []
    list_name_base = f"shift_base_{shift}"
    shift_lists_base[list_name_base] = []
    c = 0
    
    # Generate shifted chord progressions
    for i in range(len(df['chords'])):
        if c % 1000 == 0:
            print(f"Processing row {c}")
        c += 1
        chords = df['chords'][i].split()
        
        # Apply chord shift transformation
        converted_progression, converted_progression_base = convert_chord(chords, shift)
        shift_lists[list_name].append(' '.join(converted_progression))
        shift_lists_base[list_name_base].append(' '.join(converted_progression_base))

    # Create a new DataFrame with 'id' as the first column and the shifted chord progression as the second column
    new_df = pd.DataFrame({
        'id': df['id'],
        f"shift_base_{shift}": shift_lists_base[list_name_base]
    })
    
    # Save each new DataFrame to a separate CSV file
    new_df.to_csv(f'output_shifted_{shift}.csv', index=False)